In [1]:
import pandas as pd 
import numpy as np 
import re
import json 

In [2]:
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

What is the state breakdown of the gendered phrases?

In [3]:
masculine_themed_wording = [
    "competitive",
    "dominate",
    "leader",
    "rock star",
    "rockstar",
    "guru",
    "ninja",
    "hacker",
    "superhero",
    "prove themselves",
    "analyze",
    "determine",
    "crush it",
    "world class",
    "superior",
    "ambitious",
    "aggressive",
    "leader"
]

In [4]:
data_set = pd.read_csv("../data/dice/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
data_set = data_set.get(['jobtitle','jobdescription','joblocation_address'])
states = pd.read_csv("states.csv")

In [55]:
data_set['State'] = data_set['joblocation_address'].apply(lambda x: pd.Series(str(x).split(", ")[-1]))

In [56]:
descriptionDict = createTokenizedJobDescriptionDict(data_set)

In [6]:
ps = PorterStemmer()

In [15]:
def numOfOcurrences(tokenizedPhrase, tokenizedDescription): 
#     counter = 0
    # if its a phrase just look for the exact phrase in the description otherwise look at the stem of the words
#     if len(word_tokenize(phrase)) > 1: 
#         regex = re.compile(phrase)
#         return len(re.findall(regex, description))
    
#     desc = word_tokenize(description) 
#     desc = list(map(lambda x: ps.stem(x), desc))
#     return desc.count(stem)

    counter = 0
    i = len(tokenizedPhrase)
    for x in range(len(tokenizedDescription)-i + 1):
        if tokenizedDescription[x:x+i] == tokenizedPhrase:
            counter += 1; 
    return counter

In [8]:
def stem_tokens(s): 
    ps = PorterStemmer()
    tokens = word_tokenize(s)
    stem = list(map(lambda x : ps.stem(x), tokens))
    
    return stem 

In [59]:
def indicesOfJobsWithPhrase(phrase, df, descriptionDict): 
    indices = []
    for index, row in df.iterrows(): 
        tokenizedPhrase = stem_tokens(phrase)
#         tokenizedDesc = stem_tokens(row.get("jobdescription"))
        if numOfOcurrences(tokenizedPhrase, descriptionDict[index]) > 0: 
            indices.append(index)
    return indices

In [48]:
description = "IntelliFarms designs and manufactures embeddedIoT products and software that goes into agricultural applications from grainstorage and management - to weather data and automation in the fields. Ourcustomers range from Farmers to Processors you’d recognize on the back of a boxin the grocery store. We are well past our start-up phase and don’t rely onoutside capital. We’ve been on thecutting edge of AgTech for over 10 years - and need your help to stay that way.At our core, we’re Ag IoT: from sensordevelopment to monitoring and control systems, all sending data to our cloudplatform. You will learn our applicationand understand the science of what we do. You will then be responsible for all embedded software for theseproducts, from low-level drivers and hardware abstraction, to local database,algorithm, and protocol design. We workquickly, from concept to market in a short time. We range from bit-banging on 16-bitprocessors to multi-threaded RTOS design, to SQL database design, andbeyond. You should be comfortable andhave experience across all levels, primarily in C/C++.Salary range: $60K - $100K, DOEWhat we look for:Quick learner - You love to learn and are able to quickly pick up tools and put them to useExcellent at communicating your ideas and gathering requirementsDetail focused - while understanding the importance of getting a product shippedA relevant college education and/or equivalent experienceA focus on unit and system-level testingExpertise in C++, and experience with Python, SQL, and other languages.Expertise in Embedded development tool chains and debugging.At IntelliFarms you will:Solve unique problems that don't exist in other domainsBe part of a team that cares about who you are and your growth as a software engineerDesign, build, and maintain software that provides profit to farmers and food to your familyReceive great benefits (401k match, catered lunches, sushi dinners, beer fridge, dog friendly workplace, gym equipped with Pelotons, and more…)If you think we might be a match for you, please respond with your resume attached.Job Type: Full-timeSalary: $60,000.00 to $100,000.00 /yearWork Location:One location"
phrase = 'Intellifarms'
tokenizedPhrase = stem_tokens(phrase)
tokenizedDesc = stem_tokens(description)
import time
startTime = time.time()
print(numOfOcurrences(tokenizedPhrase,tokenizedDesc))
runtime = time.time() - startTime
runtime * 20 * 22000

2


311.4604949951172

In [46]:
def createTokenizedJobDescriptionDict(df): 
    descriptions = {}
    for index, row in df.iterrows(): 
        descriptions[index] = stem_tokens(row.get("jobdescription"))
    return descriptions

In [95]:
def computeStateInfo(state, full_df,descriptions): 
    stateInfo = {}
    stateInfo['region'] = state
    stateInfo['word_count'] = {}
    state_df = full_df.where(full_df['State'] == state).dropna()
    biasedIndices = []; 
    
    for word in masculine_themed_wording: 
        
        indices = indicesOfJobsWithPhrase(word, state_df, descriptions)
        stateInfo['word_count'][word] = len(indices)
        biasedIndices += indices
    
    topWords = [item[0] for item in reversed(sorted(stateInfo['word_count'].items(), key = lambda kv: kv[1]))][:5]
    
    stateInfo['numberOfJobsInDataSet'] = len(state_df)
    stateInfo['numberOfBiasedJobs'] = len(set(biasedIndices))
    try: 
        stateInfo['percentageOfBiasedJobs'] = float(stateInfo['numberOfBiasedJobs']/ stateInfo['numberOfJobsInDataSet']) * 100
    except ZeroDivisionError: 
        stateInfo['percentageOfBiasedJobs'] = 'NA'
        #     stateInfo['biasedIndices'] = list(set(biasedIndices))
                                                                                                               
    stateInfo['questionFour'] = topWords
    del(stateInfo['word_count'])
    return stateInfo
    

In [93]:
def exportInfo(DF): 
    data = [computeStateInfo(state, DF,descriptionDict) for state in states['Code']]
    stateData = {"data": data}; 
    with open('stateData.json', 'w') as outfile:
        json.dump(stateData, outfile)
    
        

In [96]:
import time

startTime = time.time()
exportInfo(data_set)
runtime = time.time() - startTime
runtime

124.61947178840637

In [77]:
D = {"region": "CA", "word_count": {"competitive": 403, "dominate": 4, "leader": 430, "rock star": 12, "rockstar": 2, "guru": 4, "ninja": 7, "hacker": 8, "superhero": 0, "prove themselves": 0, "analyze": 620, "determine": 328, "crush it": 0, "world class": 39, "superior": 105, "ambitious": 24, "aggressive": 47}, "numberOfJobsInDataSet": 4621, "numberOfBiasedJobs": 1525, "percentageOfBiasedJobs": 0.3300151482363125}

In [89]:
topWords

['analyze', 'leader', 'competitive', 'determine', 'superior']